In [1]:
import json
from src.surquest.utils.appstoreconnect.credentials import Credentials
from src.surquest.utils.appstoreconnect.analyticsreports.client import Client
from src.surquest.utils.appstoreconnect.analyticsreports.handler import Handler
from src.surquest.utils.appstoreconnect.analyticsreports.enums.category import Category
from src.surquest.utils.appstoreconnect.analyticsreports.enums.granularity import Granularity
from src.surquest.utils.appstoreconnect.analyticsreports.enums.report_name import ReportName
from pathlib import Path

In [2]:
key_path = Path.cwd() / "credentials" / "key.p8"
CATEGORY = Category.FRAMEWORK_USAGE
REPORT_NAME = ReportName.APP_OPT_IN
GRANULARITY = Granularity.DAILY
APP_ID = "950949627"

KEY_ID = '5WDUV3USAU' # 10-character key ID from App Store Connect
ISSUER_ID = '69a6de80-fd44-47e3-e053-5b8c7c11a4d1'  # 36-character issuer ID
PRIVATE_KEY = key_path.read_text()
CSV_PATH = Path.cwd() / "data" / REPORT_NAME.name / f"data.{APP_ID}.csv"
JSON_PATH = Path.cwd() / "data" / REPORT_NAME.name / f"data.{APP_ID}.jsonl"

credentials = Credentials(
    issuer_id=ISSUER_ID, # 36-character issuer ID
    key_id=KEY_ID, # 10-character key ID from App Store Connect
    private_key=PRIVATE_KEY
)

client = Client(credentials=credentials)

data = client.get_data(
    app_id = APP_ID,
    report_name = REPORT_NAME
)

Handler.list_of_dicts_to_csv(data, CSV_PATH)
Handler.list_of_dicts_to_jsonl(data, JSON_PATH)

INFO     - 2025-08-05 08:51:40,830 - Initialized Client with provided credentials
DEBUG    - 2025-08-05 08:51:40,856 - GET https://api.appstoreconnect.apple.com/v1/apps/950949627/analyticsReportRequests | Params: None
Bad pipe message: %s [b'\x8d\x06\xcf\xfe\xc22rc\r\x08\x19\xbe\xc0\x9fZ\xe4\xa2\xf7 \xa0T\x93\x00\xbaG\x19Ku\x902\x18s;x\x99\x94Y\x1d\xcc\xbe\x9f\xf1.\xd7\xfe\xb0_db?\x86\x00\x1a\xc0+\xc0/\xc0,\xc00\xcc\xa9\xcc\xa8\xc0\t\xc0\x13\xc0\n\xc0\x14\x13\x01\x13\x02\x13\x03\x01\x00\x05A\x00\x0b\x00\x02\x01\x00\xff\x01\x00\x01\x00\x00\x17\x00\x00\x00\x12\x00\x00\x00\x05\x00\x05\x01\x00\x00\x00\x00\x00\n\x00\x0c\x00\n\x11\xec\x00\x1d\x00\x17\x00\x18\x00\x19\x00\r\x00\x1a\x00\x18\x08\x04\x04\x03\x08\x07\x08\x05\x08\x06\x04\x01\x05\x01\x06\x01\x05\x03\x06\x03\x02\x01\x02\x03\x00+\x00\x05\x04\x03\x04\x03\x03\x003\x04\xea\x04\xe8\x11\xec\x04\xc0\x9a\\5\x8a\x0c\x1c{\x01Bz\xc6\xce;\xe8)y\xf5', b'\x06\x82+\x05\x8a\xf6D\xcd\xe2\xd6\x07\x99\t0v\x15\xa8\x9dW\x1b\xc5\xa4\xb4V\x85G\xcb\xd85f\xa

In [ ]:
reports = Handler.extract_attribute_values(client.list_reports(
    app_id=APP_ID
))

for i in reports:
    print(f"{i['name'].upper().replace(' ', '_')} = (\"{i['name']}\",  Category.{i['category']})")

In [ ]:
from enum import Enum

class App(str, Enum):

    LRT = "950949627"
    LGO = "1207764294"
    MV = "6451202232"

for app in [App.LRT, App.LGO, App.MV]:

    report_requests = client.read_report_requests(
        app_id=app.value
    )

    print(f"{app.name}: report requests {len(report_requests)} - {report_requests}")


    if len(report_requests) == 0:

        client.create_report_request(app_id = app.value)

In [ ]:
app = "6451202232"
report_requests = client.read_report_requests(
    app_id=app
)

report_requests
print(report_requests)
report_requests_id = report_requests[0].get("id")

specific_request = client.read_report_for_specific_request(
    report_requests_id,
    params={
        "filter[category]": REPORT_NAME.category.value,
        "filter[name]":REPORT_NAME.value
    }
)
print(Handler.extract_attribute_values(specific_request))
specific_request_id = specific_request[0].get("id")
specific_request_ids = Handler.extract_ids(specific_request)

for i in specific_request_ids:

    instances = client.read_list_of_instances_of_report(
        specific_request_id,
        params={"filter[granularity]": "DAILY"}
    )

    for inst in instances:
        print(inst)
        inst_id = inst.get('id')

        response_url = client._fetch_segment_urls([inst_id])
        print(response_url)

In [ ]:
#1A Read Report Requests
report_requests = client.read_report_requests(app_id=APP_ID)
report_request_id = report_requests.get("data")[0].get("id")
RenderJSON(report_requests)


In [ ]:
#2A Read Report
report = client.read_report_for_specific_request(
    request_id=report_request_id,
    params={
        "filter[category]": CATEGORY.value,
        "filter[name]": REPORT_NAME
    }
)

report_id = report.get("data")[0].get("id")
print(report_id)
print(report)
RenderJSON(report)

In [ ]:
#3A Read Report Instances
instances = client.read_list_of_instances_of_report(
    report_id=report_id,
    params={
        "filter[granularity]": GRANULARITY.value,
        "filter[processingDate]": DATE
    }
)
instance_id = instances.get("data")[0].get("id")
RenderJSON(instances)

In [ ]:
#4 Get segments
segments = client.read_segments_for_report(
    instance_id=instance_id
)
url  = segments.get("data")[0].get("attributes").get("url")
RenderJSON(segments)


In [ ]:
# Download the report data
data = client.download_report_to_dicts(
    report_url=url
)
data

In [ ]:
client.get_data(
    app_id = APP_ID,
    report_name = REPORT_NAME,
    category = CATEGORY,
    granularity = GRANULARITY,
    dates = {'2025-07-25', '2025-07-26'}
)
